# Manipulating the Pandas DataFrame

The Jupyter notebook for this demo can be found in:
   - docs/quick_start/demo/op2_pandas_unstack.ipynb
   - https://github.com/SteveDoyle2/pyNastran/tree/master/docs/quick_start/demo/op2_pandas_unstack.ipynb
   
### This example will use pandas unstack
The unstack method on a DataFrame moves on index level from rows to columns.  First let's read in some data:

In [1]:
import os
import pyNastran
pkg_path = pyNastran.__path__[0]
from pyNastran.op2.op2 import read_op2
import pandas as pd
pd.set_option('precision', 2)

op2_filename = os.path.join(pkg_path, '..', 'models', 'iSat', 'iSat_launch_100Hz.op2')
from pyNastran.op2.op2 import read_op2
isat = read_op2(op2_filename, build_dataframe=True, debug=False, skip_undefined_matrices=True)

INFO:      fname=op2_scalar.py             lineNo=1176   op2_filename = 'f:\\work\\pynastran\\pynastran\\master3\\pyNastran\\..\\models\\iSat\\iSat_launch_100Hz.op2'
INFO:      fname=fortran_format.py         lineNo=321    skipping table_name = 'RAPCONS'
INFO:      fname=fortran_format.py         lineNo=321    skipping table_name = 'RAQCONS'
INFO:      fname=fortran_format.py         lineNo=321    skipping table_name = 'RASCONS'
INFO:      fname=fortran_format.py         lineNo=321    skipping table_name = 'RAFCONS'
INFO:      fname=fortran_format.py         lineNo=321    skipping table_name = 'RAECONS'
INFO:      fname=fortran_format.py         lineNo=321    skipping table_name = 'RANCONS'
INFO:      fname=fortran_format.py         lineNo=321    skipping table_name = 'RAGCONS'
INFO:      fname=fortran_format.py         lineNo=321    skipping table_name = 'RAPEATC'
INFO:      fname=fortran_format.py         lineNo=321    skipping table_name = 'RAQEATC'
INFO:      fname=fortran_format.p

In [2]:
cbar = isat.cbar_force[1].data_frame

In [3]:
cbar.head()

Mode                               1         2         3         4         5   \
EigenvalueReal              2758.15   3568.63   9689.31   16168.10  16278.22    
Freq                            8.36      9.51      15.67     20.24     20.31   
Radians                        52.52     59.74     98.43     127.15    127.59   
ElementID Item                                                                  
3323      bending_moment_a1     -0.16      0.23     -1.33 -2.32e+00      1.88   
          bending_moment_a2      0.19     -0.05      0.18  5.61e-03      0.11   
          bending_moment_b1      0.17     -0.21      2.01  2.66e+00     -1.88   
          bending_moment_b2     -0.19      0.05     -0.18 -3.57e-03     -0.11   
          shear1                -0.13      0.18     -1.33 -1.99e+00      1.50   

Mode                               6         7         8         9         10  \
EigenvalueReal              16679.71  18248.43  18600.70  18632.55  32147.81    
Freq                            20.55     21.50     21.71     21.72     28.54   
Radians                        129.15    135.09    136.38    136.50    179.30   
ElementID Item                                                                  
3323      bending_moment_a1     -0.80 -1.34e-03      1.42      1.47      4.64   
          bending_moment_a2     -0.42 -4.19e-03     -1.11      0.10     -1.57   
          bending_moment_b1      0.73  2.29e-03     -1.38     -1.31     -3.97   
          bending_moment_b2      0.43  4.18e-03      1.11     -0.10      1.57   
          shear1                -0.61 -1.45e-03      1.12      1.11      3.44   

Mode                           ...           24        25        26        27  \
EigenvalueReal                 ...    253140.88 295297.75 306885.91 309040.66   
Freq                           ...        80.08     86.49     88.17     88.48   
Radians                        ...       503.13    543.41    553.97    555.91   
ElementID Item                 ...                                              
3323      bending_moment_a1    ...       -43.49     63.35    -43.08     -3.35   
          bending_moment_a2    ...        -4.46      5.33      1.63      4.86   
          bending_moment_b1    ...        34.78    -74.02     35.14      3.54   
          bending_moment_b2    ...         4.45     -5.34     -1.62     -4.86   
          shear1               ...       -31.31     54.95    -31.29     -2.76   

Mode                               28        29        30        31        32  \
EigenvalueReal              319227.72 350984.50 351566.19 364166.16 384601.34   
Freq                            89.92     94.29     94.37     96.04     98.70   
Radians                        565.00    592.44    592.93    603.46    620.16   
ElementID Item                                                                  
3323      bending_moment_a1     11.10    -14.38      0.75     29.36      0.49   
          bending_moment_a2      2.14      0.09     -1.27    -10.58     -0.67   
          bending_moment_b1    -15.06     10.97     -0.67    -17.69     -0.63   
          bending_moment_b2     -2.15     -0.08      1.27     10.56      0.67   
          shear1                10.47    -10.14      0.57     18.82      0.44   

Mode                               33  
EigenvalueReal              386090.44  
Freq                            98.89  
Radians                        621.36  
ElementID Item                         
3323      bending_moment_a1     -4.56  
          bending_moment_a2      3.48  
          bending_moment_b1      6.39  
          bending_moment_b2     -3.48  
          shear1                -4.38  

[5 rows x 33 columns]

First I'm going to pull out a small subset to work with

In [4]:
csub = cbar.loc[3323:3324,1:2]
csub

Mode                              1       2
EigenvalueReal              2758.15 3568.63
Freq                           8.36    9.51
Radians                       52.52   59.74
ElementID Item                             
3323      bending_moment_a1   -0.16    0.23
          bending_moment_a2    0.19   -0.05
          bending_moment_b1    0.17   -0.21
          bending_moment_b2   -0.19    0.05
          shear1              -0.13    0.18
          shear2               0.15   -0.04
          axial                0.80    0.21
          torque              -0.04   -0.06
3324      bending_moment_a1    0.14   -0.29
          bending_moment_a2   -0.19    0.05
          bending_moment_b1   -0.15    0.26
          bending_moment_b2    0.19   -0.05
          shear1               0.12   -0.22
          shear2              -0.15    0.04
          axial               -0.80   -0.21
          torque               0.04    0.06

I happen to like the way that's organized, but let's say that I want the have the item descriptions in columns and the mode ID's and element numbers in rows.  To do that, I'll first move the element ID's up to the columns using a .unstack(level=0) and the transpose the result:

In [5]:
csub.unstack(level=0).T

Item                                        bending_moment_a1  \
Mode EigenvalueReal Freq Radians ElementID                      
1    2758.15        8.36 52.52   3323                   -0.16   
                                 3324                    0.14   
2    3568.63        9.51 59.74   3323                    0.23   
                                 3324                   -0.29   

Item                                        bending_moment_a2  \
Mode EigenvalueReal Freq Radians ElementID                      
1    2758.15        8.36 52.52   3323                    0.19   
                                 3324                   -0.19   
2    3568.63        9.51 59.74   3323                   -0.05   
                                 3324                    0.05   

Item                                        bending_moment_b1  \
Mode EigenvalueReal Freq Radians ElementID                      
1    2758.15        8.36 52.52   3323                    0.17   
                                 3324                   -0.15   
2    3568.63        9.51 59.74   3323                   -0.21   
                                 3324                    0.26   

Item                                        bending_moment_b2  shear1  shear2  \
Mode EigenvalueReal Freq Radians ElementID                                      
1    2758.15        8.36 52.52   3323                   -0.19   -0.13    0.15   
                                 3324                    0.19    0.12   -0.15   
2    3568.63        9.51 59.74   3323                    0.05    0.18   -0.04   
                                 3324                   -0.05   -0.22    0.04   

Item                                        axial  torque  
Mode EigenvalueReal Freq Radians ElementID                 
1    2758.15        8.36 52.52   3323        0.80   -0.04  
                                 3324       -0.80    0.04  
2    3568.63        9.51 59.74   3323        0.21   -0.06  
                                 3324       -0.21    0.06

unstack requires unique row indices so I can't work with CQUAD4 stresses as they're currently output, but I'll work with CHEXA stresses.  Let's pull out the first two elements and first two modes:

In [6]:
chs = isat.chexa_stress[1].data_frame.loc[3684:3685,1:2]
chs

Mode                               1         2
EigenvalueReal               2758.15   3568.63
Freq                            8.36      9.51
Radians                        52.52     59.74
ElementID NodeID Item                         
3684      0      oxx        7.93e-12  1.14e-13
                 oyy       -3.58e-12 -7.96e-13
                 ozz       -3.41e-13  2.27e-13
                 txy       -1.99e-13  0.00e+00
                 tyz        2.84e-14  5.68e-14
                 txz        5.12e-13 -2.27e-13
                 omax       7.96e-12  4.07e-13
                 omid      -3.72e-13 -6.22e-14
                 omin      -3.59e-12 -7.99e-13
                 von_mises  1.03e-11  1.05e-12
          55     oxx        2.33e-12 -2.10e-12
                 oyy        1.18e-12 -1.93e-12
                 ozz       -9.45e-13 -1.05e-12
                 txy       -3.98e-13 -9.09e-13
                 tyz       -2.62e-14 -9.88e-14
                 txz       -4.76e-13 -6.53e-13
                 omax       2.51e-12 -6.34e-13
                 omid       1.07e-12 -1.37e-12
                 omin      -1.02e-12 -3.08e-12
                 von_mises  3.07e-12  2.18e-12
          51     oxx       -8.67e-13 -3.98e-13
                 oyy        2.29e-12  9.66e-13
                 ozz       -2.53e-12  7.11e-13
                 txy       -3.15e-13 -1.98e-12
                 tyz        2.17e-14 -3.56e-13
                 txz        1.95e-13 -1.58e-13
                 omax       2.32e-12  2.40e-12
                 omid      -8.75e-13  7.31e-13
                 omin      -2.55e-12 -1.85e-12
                 von_mises  4.29e-12  3.71e-12
...                              ...       ...
3685      46     oxx       -2.38e-13  2.84e-13
                 oyy        6.68e-13 -3.41e-13
                 ozz        1.28e-13  1.14e-13
                 txy        1.05e-13  3.84e-16
                 tyz       -2.84e-14  0.00e+00
                 txz        2.14e-13 -9.59e-14
                 omax       6.80e-13  3.27e-13
                 omid       2.26e-13  7.06e-14
                 omin      -3.48e-13 -3.41e-13
                 von_mises  8.92e-13  5.84e-13
          1031   oxx       -2.74e-12 -3.41e-13
                 oyy       -5.68e-13  3.41e-13
                 ozz       -1.42e-12 -4.55e-13
                 txy        1.07e-13  0.00e+00
                 tyz        0.00e+00 -2.27e-13
                 txz       -4.55e-13  4.55e-13
                 omax      -5.63e-13  4.26e-13
                 omid      -1.28e-12 -1.01e-28
                 omin      -2.89e-12 -8.80e-13
                 von_mises  2.06e-12  1.15e-12
          1037   oxx        2.42e-13 -1.14e-13
                 oyy       -1.28e-13 -4.55e-13
                 ozz        1.14e-13  0.00e+00
                 txy       -7.11e-15  2.84e-13
                 tyz       -1.42e-14 -1.14e-13
                 txz       -2.27e-13  4.55e-13
                 omax       4.14e-13  4.15e-13
                 omid      -5.52e-14 -2.05e-13
                 omin      -1.31e-13 -7.79e-13
                 von_mises  5.11e-13  1.03e-12

[180 rows x 2 columns]

Now I want to put ElementID and the Node ID in the rows along with the Load ID, and have the items in the columns:

In [7]:
cht = chs.unstack(level=[0,1]).T
cht

Item                                                    oxx       oyy  \
Mode EigenvalueReal Freq Radians ElementID NodeID                       
1    2758.15        8.36 52.52   3684      0       7.93e-12 -3.58e-12   
                                           41           NaN       NaN   
                                           45           NaN       NaN   
                                           46           NaN       NaN   
                                           50           NaN       NaN   
                                           51     -8.67e-13  2.29e-12   
                                           55      2.33e-12  1.18e-12   
                                           56     -5.37e-12 -2.76e-12   
                                           60      8.81e-13 -2.10e-12   
                                           758     2.22e-12  1.63e-12   
                                           778    -3.24e-12 -1.88e-12   
                                           880     1.76e-12  1.88e-12   
                                           952    -5.29e-12 -7.11e-13   
                                           1015         NaN       NaN   
                                           1021         NaN       NaN   
                                           1031         NaN       NaN   
                                           1037         NaN       NaN   
                                 3685      0       7.18e-13  1.85e-13   
                                           41     -8.46e-13 -6.04e-13   
                                           45      1.06e-12 -5.65e-13   
                                           46     -2.38e-13  6.68e-13   
                                           50     -6.04e-14  1.14e-13   
                                           51           NaN       NaN   
                                           55           NaN       NaN   
                                           56           NaN       NaN   
                                           60           NaN       NaN   
                                           758          NaN       NaN   
                                           778          NaN       NaN   
                                           880          NaN       NaN   
                                           952          NaN       NaN   
...                                                     ...       ...   
2    3568.63        9.51 59.74   3684      50           NaN       NaN   
                                           51     -3.98e-13  9.66e-13   
                                           55     -2.10e-12 -1.93e-12   
                                           56      2.84e-13 -4.55e-13   
                                           60      5.12e-13  1.14e-12   
                                           758     3.41e-13  2.05e-12   
                                           778     2.27e-13  2.27e-13   
                                           880    -1.71e-12  1.59e-12   
                                           952    -1.02e-12  1.36e-12   
                                           1015         NaN       NaN   
                                           1021         NaN       NaN   
                                           1031         NaN       NaN   
                                           1037         NaN       NaN   
                                 3685      0       1.14e-13 -3.41e-13   
                                           41     -2.84e-13  2.27e-13   
                                           45     -5.68e-13  0.00e+00   
                                           46      2.84e-13 -3.41e-13   
                                           50      5.12e-13  6.82e-13   
                                           51           NaN       NaN   
                                           55           NaN       NaN   
                                           56           NaN       NaN   
                                           60           Na

Maybe I'd like my rows organized with the modes on the inside.  I can do that by swapping levels:

We actually need to get rid of the extra rows using dropna():

In [8]:
cht = cht.dropna()
cht

Item                                                    oxx       oyy  \
Mode EigenvalueReal Freq Radians ElementID NodeID                       
1    2758.15        8.36 52.52   3684      0       7.93e-12 -3.58e-12   
                                           51     -8.67e-13  2.29e-12   
                                           55      2.33e-12  1.18e-12   
                                           56     -5.37e-12 -2.76e-12   
                                           60      8.81e-13 -2.10e-12   
                                           758     2.22e-12  1.63e-12   
                                           778    -3.24e-12 -1.88e-12   
                                           880     1.76e-12  1.88e-12   
                                           952    -5.29e-12 -7.11e-13   
                                 3685      0       7.18e-13  1.85e-13   
                                           41     -8.46e-13 -6.04e-13   
                                           45      1.06e-12 -5.65e-13   
                                           46     -2.38e-13  6.68e-13   
                                           50     -6.04e-14  1.14e-13   
                                           1015   -5.68e-14 -7.85e-13   
                                           1021    9.66e-13  2.17e-13   
                                           1031   -2.74e-12 -5.68e-13   
                                           1037    2.42e-13 -1.28e-13   
2    3568.63        9.51 59.74   3684      0       1.14e-13 -7.96e-13   
                                           51     -3.98e-13  9.66e-13   
                                           55     -2.10e-12 -1.93e-12   
                                           56      2.84e-13 -4.55e-13   
                                           60      5.12e-13  1.14e-12   
                                           758     3.41e-13  2.05e-12   
                                           778     2.27e-13  2.27e-13   
                                           880    -1.71e-12  1.59e-12   
                                           952    -1.02e-12  1.36e-12   
                                 3685      0       1.14e-13 -3.41e-13   
                                           41     -2.84e-13  2.27e-13   
                                           45     -5.68e-13  0.00e+00   
                                           46      2.84e-13 -3.41e-13   
                                           50      5.12e-13  6.82e-13   
                                           1015   -1.14e-13  5.68e-14   
                                           1021    0.00e+00  5.68e-14   
                                           1031   -3.41e-13  3.41e-13   
                                           1037   -1.14e-13 -4.55e-13   

Item                                                    ozz       txy  \
Mode EigenvalueReal Freq Radians ElementID NodeID                       
1    2758.15        8.36 52.52   3684      0      -3.41e-13 -1.99e-13   
                                           51     -2.53e-12 -3.15e-13   
                                           55     -9.45e-13 -3.98e-13   
                                           56     -2.43e-12 -4.92e-13   
                                           60     -4.97e-13  5.68e-14   
                                           758     2.43e-12 -1.99e-13   
                                           778    -4.99e-12 -1.15e-13   
                                           880     1.82e-12 -5.26e-13   
                                           952    -1.93e-12 -1.71e-13   
                                 3685      0      -2.13e-13  1.17e-13   
                                           41     -6.04e-14  2.20e-13   
                                           45      5.68e-13  3.55e-14   
                                           46      1.28e-13  1.05e-13   
                                           50      6.39e-14 -4.26e-14   
                                           1015   -2.70e-

In [9]:
# mode, eigr, freq, rad, eids, nids # initial
# nids, eids, eigr, freq, rad, mode # final

cht.swaplevel(0,4).swaplevel(1,5).swaplevel(2,5).swaplevel(4, 5)

Item                                                    oxx       oyy  \
ElementID NodeID EigenvalueReal Radians Freq Mode                       
3684      0      2758.15        52.52   8.36 1     7.93e-12 -3.58e-12   
          51     2758.15        52.52   8.36 1    -8.67e-13  2.29e-12   
          55     2758.15        52.52   8.36 1     2.33e-12  1.18e-12   
          56     2758.15        52.52   8.36 1    -5.37e-12 -2.76e-12   
          60     2758.15        52.52   8.36 1     8.81e-13 -2.10e-12   
          758    2758.15        52.52   8.36 1     2.22e-12  1.63e-12   
          778    2758.15        52.52   8.36 1    -3.24e-12 -1.88e-12   
          880    2758.15        52.52   8.36 1     1.76e-12  1.88e-12   
          952    2758.15        52.52   8.36 1    -5.29e-12 -7.11e-13   
3685      0      2758.15        52.52   8.36 1     7.18e-13  1.85e-13   
          41     2758.15        52.52   8.36 1    -8.46e-13 -6.04e-13   
          45     2758.15        52.52   8.36 1     1.06e-12 -5.65e-13   
          46     2758.15        52.52   8.36 1    -2.38e-13  6.68e-13   
          50     2758.15        52.52   8.36 1    -6.04e-14  1.14e-13   
          1015   2758.15        52.52   8.36 1    -5.68e-14 -7.85e-13   
          1021   2758.15        52.52   8.36 1     9.66e-13  2.17e-13   
          1031   2758.15        52.52   8.36 1    -2.74e-12 -5.68e-13   
          1037   2758.15        52.52   8.36 1     2.42e-13 -1.28e-13   
3684      0      3568.63        59.74   9.51 2     1.14e-13 -7.96e-13   
          51     3568.63        59.74   9.51 2    -3.98e-13  9.66e-13   
          55     3568.63        59.74   9.51 2    -2.10e-12 -1.93e-12   
          56     3568.63        59.74   9.51 2     2.84e-13 -4.55e-13   
          60     3568.63        59.74   9.51 2     5.12e-13  1.14e-12   
          758    3568.63        59.74   9.51 2     3.41e-13  2.05e-12   
          778    3568.63        59.74   9.51 2     2.27e-13  2.27e-13   
          880    3568.63        59.74   9.51 2    -1.71e-12  1.59e-12   
          952    3568.63        59.74   9.51 2    -1.02e-12  1.36e-12   
3685      0      3568.63        59.74   9.51 2     1.14e-13 -3.41e-13   
          41     3568.63        59.74   9.51 2    -2.84e-13  2.27e-13   
          45     3568.63        59.74   9.51 2    -5.68e-13  0.00e+00   
          46     3568.63        59.74   9.51 2     2.84e-13 -3.41e-13   
          50     3568.63        59.74   9.51 2     5.12e-13  6.82e-13   
          1015   3568.63        59.74   9.51 2    -1.14e-13  5.68e-14   
          1021   3568.63        59.74   9.51 2     0.00e+00  5.68e-14   
          1031   3568.63        59.74   9.51 2    -3.41e-13  3.41e-13   
          1037   3568.63        59.74   9.51 2    -1.14e-13 -4.55e-13   

Item                                                    ozz       txy  \
ElementID NodeID EigenvalueReal Radians Freq Mode                       
3684      0      2758.15        52.52   8.36 1    -3.41e-13 -1.99e-13   
          51     2758.15        52.52   8.36 1    -2.53e-12 -3.15e-13   
          55     2758.15        52.52   8.36 1    -9.45e-13 -3.98e-13   
          56     2758.15        52.52   8.36 1    -2.43e-12 -4.92e-13   
          60     2758.15        52.52   8.36 1    -4.97e-13  5.68e-14   
          758    2758.15        52.52   8.36 1     2.43e-12 -1.99e-13   
          778    2758.15        52.52   8.36 1    -4.99e-12 -1.15e-13   
          880    2758.15        52.52   8.36 1     1.82e-12 -5.26e-13   
          952    2758.15        52.52   8.36 1    -1.93e-12 -1.71e-13   
3685      0      2758.15        52.52   8.36 1    -2.13e-13  1.17e-13   
          41     2758.15        52.52   8.36 1    -6.04e-14  2.20e-13   
          45     2758.15        52.52   8.36 1     5.68e-13  3.55e-14   
          46     2758.15        52.52   8.36 1     1.28e-13  1.05e-13   
          50     2758.15        52.52   8.36 1     6.39e-14 -4.26e-14   
          1015   2758.15        52.52   8.36 1    -2.70e-

Alternatively I can do that by first using reset_index to move all the index columns into data, and then using set_index to define the order of columns I want as my index:

In [10]:
cht.reset_index().set_index(['ElementID','NodeID','Mode','Freq']).sort_index()

Item                        EigenvalueReal  Radians       oxx       oyy  \
ElementID NodeID Mode Freq                                                
3684      0      1    8.36         2758.15    52.52  7.93e-12 -3.58e-12   
                 2    9.51         3568.63    59.74  1.14e-13 -7.96e-13   
          51     1    8.36         2758.15    52.52 -8.67e-13  2.29e-12   
                 2    9.51         3568.63    59.74 -3.98e-13  9.66e-13   
          55     1    8.36         2758.15    52.52  2.33e-12  1.18e-12   
                 2    9.51         3568.63    59.74 -2.10e-12 -1.93e-12   
          56     1    8.36         2758.15    52.52 -5.37e-12 -2.76e-12   
                 2    9.51         3568.63    59.74  2.84e-13 -4.55e-13   
          60     1    8.36         2758.15    52.52  8.81e-13 -2.10e-12   
                 2    9.51         3568.63    59.74  5.12e-13  1.14e-12   
          758    1    8.36         2758.15    52.52  2.22e-12  1.63e-12   
                 2    9.51         3568.63    59.74  3.41e-13  2.05e-12   
          778    1    8.36         2758.15    52.52 -3.24e-12 -1.88e-12   
                 2    9.51         3568.63    59.74  2.27e-13  2.27e-13   
          880    1    8.36         2758.15    52.52  1.76e-12  1.88e-12   
                 2    9.51         3568.63    59.74 -1.71e-12  1.59e-12   
          952    1    8.36         2758.15    52.52 -5.29e-12 -7.11e-13   
                 2    9.51         3568.63    59.74 -1.02e-12  1.36e-12   
3685      0      1    8.36         2758.15    52.52  7.18e-13  1.85e-13   
                 2    9.51         3568.63    59.74  1.14e-13 -3.41e-13   
          41     1    8.36         2758.15    52.52 -8.46e-13 -6.04e-13   
                 2    9.51         3568.63    59.74 -2.84e-13  2.27e-13   
          45     1    8.36         2758.15    52.52  1.06e-12 -5.65e-13   
                 2    9.51         3568.63    59.74 -5.68e-13  0.00e+00   
          46     1    8.36         2758.15    52.52 -2.38e-13  6.68e-13   
                 2    9.51         3568.63    59.74  2.84e-13 -3.41e-13   
          50     1    8.36         2758.15    52.52 -6.04e-14  1.14e-13   
                 2    9.51         3568.63    59.74  5.12e-13  6.82e-13   
          1015   1    8.36         2758.15    52.52 -5.68e-14 -7.85e-13   
                 2    9.51         3568.63    59.74 -1.14e-13  5.68e-14   
          1021   1    8.36         2758.15    52.52  9.66e-13  2.17e-13   
                 2    9.51         3568.63    59.74  0.00e+00  5.68e-14   
          1031   1    8.36         2758.15    52.52 -2.74e-12 -5.68e-13   
                 2    9.51         3568.63    59.74 -3.41e-13  3.41e-13   
          1037   1    8.36         2758.15    52.52  2.42e-13 -1.28e-13   
                 2    9.51         3568.63    59.74 -1.14e-13 -4.55e-13   

Item                             ozz       txy       tyz       txz      omax  \
ElementID NodeID Mode Freq                                                     
3684      0      1    8.36 -3.41e-13 -1.99e-13  2.84e-14  5.12e-13  7.96e-12   
                 2    9.51  2.27e-13  0.00e+00  5.68e-14 -2.27e-13  4.07e-13   
          51     1    8.36 -2.53e-12 -3.15e-13  2.17e-14  1.95e-13  2.32e-12   
                 2    9.51  7.11e-13 -1.98e-12 -3.56e-13 -1.58e-13  2.40e-12   
          55     1    8.36 -9.45e-13 -3.98e-13 -2.62e-14 -4.76e-13  2.51e-12   
                 2    9.51 -1.05e-12 -9.09e-13 -9.88e-14 -6.53e-13 -6.34e-13   
          56     1    8.36 -2.43e-12 -4.92e-13  1.14e-13 -1.95e-13 -2.35e-12   
                 2    9.51 -8.53e-13 -9.09e-13 -2.27e-13  9.52e-14  9.20e-13   
          60     1    8.36 -4.97e-13  5.68e-14  1.14e-13 -2.93e-13  9.41e-13   
                 2    9.51  5.12e-13 -9.09e-13 -2.27e-13 -3.68e-14  1.81e-12   
          758    1    8.36  2.43e-12 -1.99e-13 -2.49e-14 -3.41e-13  2.69e-12   
                 2    9.51  1.08e-12 -9.09e-13 -3.40e-13  4.55e-13  2.60e-12   
          778    1    8.36 -4